In [1]:
"""
Notes:
- GarageType variable might be biased. Note that it has a 2types variable making our dummy variables technically incorrect since we dont know which 2 types it is
- PavedDrive: It could be argued that this variable should be numerically categorized rather than turned into dummy variables
"""

'\nNotes:\n- GarageType variable might be biased. Note that it has a 2types variable making our dummy variables technically incorrect since we dont know which 2 types it is\n- PavedDrive: It could be argued that this variable should be numerically categorized rather than turned into dummy variables\n'

In [2]:
import numpy as np
import pandas as pd

import os

In [3]:
# SET ENV VARIABLES
data_path = '../../data'
train_csv = os.path.join(data_path, 'train.csv')
test_csv = os.path.join(data_path, 'test.csv')

In [4]:
# Read CSV
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

In [5]:
# Note the difference between columns in train_df and test_df
print(f"Test DF Unique Columns: {list(set(test_df.columns) - set(train_df.columns))} \n")
print(f"Train DF Unique Columns: {list(set(train_df.columns) - set(test_df.columns))} \n")

Test DF Unique Columns: ['BldgType', 'Exterior1st', 'Neighborhood', 'LandSlope', 'YearRemodAdd', 'RoofMatl', 'LotArea', 'LotFrontage', 'RoofStyle', 'LotShape', 'LotConfig', 'LandContour', 'HouseStyle', 'Exterior2nd', 'OverallCond', 'Street', 'YearBuilt', 'Condition1', 'MSZoning', 'MasVnrArea', 'Condition2', 'MasVnrType', 'Alley', 'OverallQual', 'Utilities'] 

Train DF Unique Columns: ['SalePrice'] 



In [6]:
# Get insight of data
train_df.head(10)

,Id,MSSubClass,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,Gd,TA,PConc,Gd,TA,No,GLQ,706,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,TA,TA,BrkTil,TA,Gd,No,ALQ,216,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,Gd,TA,PConc,Gd,TA,Av,GLQ,655,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,TA,TA,Wood,Gd,TA,No,GLQ,732,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,TA,TA,BrkTil,TA,TA,No,Unf,0,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,TA,TA,BrkTil,TA,TA,No,GLQ,851,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [7]:
# Convert Disproportionate Classification Variables

"""
NOTE: The electrical variable is disproportionately SBrkr. As a result, we will convert Electrical to a boolean of SBrkr vs Not SBrkr rather than creating more variables

SBrkr - 1334 (91.5%)
FuseA - 1    (0.05%)
FuseF - 94   (6.5%)
FuseP - 27   (1.8%)
Mix   - 3    (0.25%)

NOTE: The BsmtFinType2 variable is disproportionately SBrkr. As a result, we will convert Electrical to a boolean of SBrkr vs Not SBrkr rather than creating more variables

GLQ - 14
ALQ - 19
BLQ - 33
Rec - 54
LwQ - 46
Unf - 1256
NA - 38


Note: An argument could be made to do the same for BsmtExposure, though we have decided against converting it into a boolean variable

GD - 134
Av - 221
Mn - 114
No - 953
NA - 38

"""
disp_cats = [('Electrical', 'SBrkr'), ('BsmtFinType2', 'Unf')]

for cat, key in disp_cats:
    train_df[cat] = [1 if e == key else 0 for e in train_df[cat]]

In [8]:
# Convert Relative Categorical into numerical
rating_order_map = {
    'Po': 0,
    'Fa': 1,
    'TA': 2,
    'Gd': 3,
    'Ex': 4,
    'NA': 'NA',
    np.nan: 'NA'
}

basement_exposure_map = {
    'No': 0,
    'Mn': 1,
    'Av': 2,
    'Gd': 3,
    'NA': 'NA',
    np.nan: 'NA'
}

basement_finish_map = {
    'Unf': 0,
    'LwQ': 1,
    'Rec': 2,
    'BLQ': 3,
    'ALQ': 4,
    'GLQ': 5,
    'NA': 'NA',
    np.nan: 'NA'
}

functional_rating = {
    'Sal': 0,
    'Sev': 1,
    'Maj2': 2,
    'Maj1': 3,
    'Mod': 4,
    'Min2': 5,
    'Min1': 6,
    'Typ': 7
}

garage_finish_rating = {
    'Unf': 0,
    'RFn': 1,
    'Fin': 2,
    'NA': 'NA',
    np.nan: 'NA'
}

fence_quality_rating = {
    "MnWw": 0,
    "GdWo": 1,
    "MnPrv": 2,
    "GdPrv": 3,
    "NA": "NA",
    np.nan: 'NA'
}

bool_map = {
    'N': 0,
    'Y': 1
}

categories = (
    [(cat, rating_order_map) for cat in ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']] + 
    [(cat, basement_exposure_map) for cat in ['BsmtExposure']] +
    [(cat, basement_finish_map) for cat in ['BsmtFinType1']] +
    [(cat, functional_rating) for cat in ['Functional']] +
    [(cat, garage_finish_rating) for cat in ['GarageFinish']] +
    [(cat, fence_quality_rating) for cat in ['Fence']] +
    [(cat, bool_map) for cat in ['CentralAir']]
)

for cat, map in categories:
    num_na = len([i for i in train_df[cat] if i is np.nan])
    
    train_df[cat] = [map[rating] for rating in train_df[cat]]
    cat_na = f'{cat}_NA'
    
    # Create Boolean NA Column
    train_df[cat_na] = [1 if v == 'NA' else 0 for v in train_df[cat]]
    
    assert(num_na == sum(train_df[cat_na]))
    
    # Convert NA to average (IS THIS REALLY THE BEST STRATEGY?)
    avg = np.average([i for i in train_df[cat] if i != 'NA'])
    train_df[cat] = [i if i != 'NA' else avg for i in train_df[cat]]

In [9]:
# Convert to cyclical (See for more information)
cyclical_features = ['MoSold']
for f in cyclical_features:
    train_df[f'{f}_sin'] = np.sin(train_df[f])
    train_df[f'{f}_cos'] = np.cos(train_df[f])
    train_df.drop(f, 1)

In [10]:
# Decide which categorical variables you want to use
# TODO: Convert to bar chart
for col_name in train_df.columns:
    if train_df[col_name].dtypes == 'object':
        unique_cat = len(train_df[col_name].unique())
        print(f'Feature {col_name} has {unique_cat} unique categories')

Feature Foundation has 6 unique categories
Feature Heating has 6 unique categories
Feature GarageType has 7 unique categories
Feature PavedDrive has 3 unique categories
Feature MiscFeature has 5 unique categories
Feature SaleType has 9 unique categories
Feature SaleCondition has 6 unique categories


In [11]:
def dummy_df(df, todummy_list):
    for cat in todummy_list:
        dummies = pd.get_dummies(train_df[cat], prefix=cat, dummy_na=True)
        df = df.drop(cat, 1)
        df = pd.concat([df, dummies], axis=1)
    return df

dummy_list = [col for col in train_df.columns if train_df[col].dtypes == 'object']
dummy_list = dummy_list + ['MSSubClass']

train_df = dummy_df(train_df, dummy_list)

In [12]:
# Check if any columns are still null
cols_with_na = train_df.isnull().sum().sort_values(ascending=False)
cols_with_na = cols_with_na[cols_with_na > 0]
cols_with_na

print(list(cols_with_na.keys()))

['GarageYrBlt']


In [13]:
# NOTE: Since Garage YR blt has NA values, we have decided to deal with this by using the average year as the NA value 
# (IS THIS REALLY THE BEST STRATEGY?)
avg_garage_blt_yr = np.average([i for i in train_df['GarageYrBlt'] if not np.isnan(i)])
train_df['GarageYrBlt'] = [avg_garage_blt_yr if np.isnan(i) else i for i in train_df['GarageYrBlt']]

In [14]:
# Verify no more columns are NaN
cols_with_na = train_df.isnull().sum().sort_values(ascending=False)
cols_with_na = cols_with_na[cols_with_na > 0]
assert 0 == cols_with_na.shape[0]